# 특정단어(ex. 연남동)가 언급된 트위터 개수 **한 달씩** 크롤링하기

<br>
참고 : [왕형준님 블로그](https://medium.com/@whj2013123218/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-twitter-%ED%81%AC%EB%A1%A4%EB%A7%81-576f7b098daf) 갓형준님 감사합니다

중복되는 코드를 없앴고 연도별 데이터를 보는 우리 프로젝트에 맞게 코드를 조금 수정했습니다.

#### 먼저 코드를 실행시키기 위해서는

1) Firefox 브라우저가 깔려있어야 합니다.

2) geckodriver.exe가 현재 디렉토리에 있는지 확인해주세요.


## total_twitter_permonth 함수 설명
#### 트위터에서 제공하는 기존 twitter API의 문제점
 무료 API는 최근 7일치 트위터밖에 가져오지 못한다. <br>
 우리가 보려는 것은 2009년부터 현재까지 데이터이므로 twitter에서 제공하는 무료 API는 사용할 수 없었다.<br>
 과거 데이터를 가져오려면 상당한 액수를 지불해야 한다. (우리 프로젝트라면 $399 짜리를 써야 했다...)
<br>
####  그래서 만든것이 total_twitter_permonth함수
날짜, 지역(동), 트윗 수, 하트 수, 연도를 담아 데이터프레임으로 반환<br>
트위터에서 특정 기간동안 특정 단어를 포함한 트윗들을 검색<br>
스크롤을 내려 많은 트윗을 계속 로딩해 와야 하기 때문에 selenium으로 동적 크롤링을 했다.<br>
chrome은 느리다는 말이 있어서 firefox를 이용했고 geckodriver로 크롤링 시행했다<br>
#### 개발하면서 생긴 문제점들 
1. 트위터 크롤링 시 로딩 오류

In [47]:
%%html
<div class="js-stream-whale-end stream-whale-end stream-placeholder centered-placeholder">
  <div class="stream-end-inner">
    <h2 class="title">로딩하는데 시간이 지연되고 있습니다.</h2>
    <p>
      트위터의 트래픽이 폭주했거나 일시적인 문제가 발생했을 수 있습니다. <a role="button" href="#" class="try-again-after-whale">다시 시도</a>하거나 <a target="_blank" href="http://status.twitter.com" rel="noopener">트위터 상태 페이지</a>를 방문하여 자세한 내용을 확인해 보세요.
    </p>
  </div>
</div>

   해결방안: 위 문구가 나타나면 **트윗수(Frequency)**를 **-1**로 저장하도록 함

2. 다음 날짜로 넘어갈 때 오류가 나서 datetime.timedelta대신 relativedelta 라이브러리 사용

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import  FirefoxBinary
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities
import time
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import re
import warnings
warnings.filterwarnings('ignore')

In [37]:
def total_twitter_permonth(keyword,startyear,endyear):
    binary=FirefoxBinary('C:/Program Files/Mozilla Firefox/firefox.exe')
    browser=webdriver.Firefox(executable_path='geckodriver.exe',firefox_binary=binary)
    
    df = pd.DataFrame(data={'Date':[],'Frequency':[],'Heart':[],'Year':[],'Dong':[]})
    
    for loc in keyword:
        print(loc)
        for y in range(startyear,endyear+1):
            startdate = dt.date(year=y,month=1,day=1)
            untildate = dt.date(year=y,month=2,day=1)

            if y == 2018 :
                enddate = dt.date(year=y,month=9,day=1)
            else :
                enddate = dt.date(year=y+1,month=1,day=1)

            totalfreq=[]
            while not enddate == startdate:
                url='https://twitter.com/search?&q="'+str(loc)+'"%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'
                browser.get(url)

                lastHeight = browser.execute_script("return document.body.scrollHeight")  # 스크롤 내리기 전 위치
                while True:
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 스크롤을 맨 아래까지 내림
                    time.sleep(4)  # 로딩하는 시간때문에 sleep을 걸어줍니다.

                    newHeight = browser.execute_script("return document.body.scrollHeight")  # 현재 스크롤 위치

                    if newHeight == lastHeight:  # 더이상 로딩될 트윗이 없다면
                        html = browser.page_source
                        soup = BeautifulSoup(html,'html.parser')
                                                                  
                        tweets = soup.find_all("p", {"class": "TweetTextSize"})
                        wordfreq = len(tweets)  # 페이지에 있는 트윗개수

                        heartfreq = 0
                        hearts = soup.find_all("span",text = re.compile("\d 마음"))
                        for hf in hearts:
                            heartfreq += int(hf.text[:-3].replace(',',''))

                        if(len(soup.find_all("h2",text = re.compile("로딩하는데 시간이 지연되고 있습니다."))) == 1):
                            wordfreq = -1
                            
                        dailyfreq = {'Date':startdate,'Frequency':wordfreq,'Heart':heartfreq,'Year':startdate.year,'Dong':loc}
                        totalfreq.append(dailyfreq)

                        # 다음 작업을 위해 하루 늘려줌                  
                        startdate = untildate
                        untildate += relativedelta(months=1)
                        # 초기화
                        wordfreq = 0
                        dailyfreq = {}
                        break
                    lastHeight = newHeight

            df = pd.concat([df,pd.DataFrame(totalfreq)])
    return df

## 본격 서울시 467개동 크롤링

기존 공시지가 데이터에서 서울시 법정동 이름만 추출<br>
XX1가동 XX1동 과 같은 경우는 해당 트윗이 많이 없을것이라 예상하고 XX동으로 전처리를 시켜주었다.
총 358개동이 추출됨

In [45]:
land = pd.read_csv('Report.csv',encoding='cp949')

In [46]:
dong_list = []
for d in land['법정동'].unique()[3:]:
    d = re.sub('\d가','',d)
    d = re.sub('\d동','동',d)
    dong_list.append(d)
dong_list = list(set(dong_list))

In [5]:
len(dong_list)

358

In [11]:
for n in range(1,71):
    df = total_twitter_permonth(dong_list[5*n:5*(n+1)],2009,2017)
    df.to_csv('twit_seoul/seoul_dong_'+str(n)+'.csv',encoding='utf-8-sig',index=False)

염곡동


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



내발산동
관수동
번동
교북동
이태원동
여의도동
보문동


In [13]:
df = total_twitter_permonth(dong_list[355:],2009,2017)
df.to_csv('twit_seoul/seoul_dong_72.csv',encoding='utf-8-sig',index=False)

이태원동


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



여의도동
보문동


In [15]:
df = total_twitter_permonth(['연남동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_yn.csv',encoding='utf-8-sig',index=False)

연남동


In [ ]:
df = total_twitter_permonth(['익선동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_is.csv',encoding='utf-8-sig',index=False)

In [6]:
df = total_twitter_permonth(['후암동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_ha.csv',encoding='utf-8-sig',index=False)

후암동


In [ ]:
df = total_twitter_permonth(['성수동'],2015,2017)
df.to_csv('twit_seoul/seoul_dong_ss.csv',encoding='utf-8-sig',index=False)

In [10]:
df = total_twitter_permonth(['봉천동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_bc.csv',encoding='utf-8-sig',index=False)

봉천동


In [11]:
df = total_twitter_permonth(['망원동'],2009,2017)
df.to_csv('twit_seoul/seoul_dong_mw.csv',encoding='utf-8-sig',index=False)

망원동
